In [50]:
from langchain.output_parsers import (CommaSeparatedListOutputParser,DatetimeOutputParser)
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import (OpenAI,ChatOpenAI)
import os

from langchain.chains import LLMChain

# 创建一个输出解析器，用于处理带逗号分隔的列表输出
output_parser = CommaSeparatedListOutputParser()

# 获取格式化指令，该指令告诉模型如何格式化其输出
format_instructions = output_parser.get_format_instructions()


MODEL_NAME = "gpt-3.5-turbo"

llm = OpenAI(model='davinci-002',openai_proxy=os.getenv("OPENAI_BASE_URL"),max_tokens=1000)

chat_model  = ChatOpenAI(model=MODEL_NAME,openai_proxy=os.getenv("OPENAI_BASE_URL"),max_tokens=1000)




In [53]:

# 创建一个提示模板，它会基于给定的模板和变量来生成提示
template = ChatPromptTemplate.from_messages(messages=[
    ("system", "You are a helpful AI bot"),
    ("human", "List five {subject}.\n{format_instructions}"),
])

# 使用提示模板和给定的主题来格式化输入
messages = template.format_messages(subject="ice cream flavors",format_instructions=format_instructions)

template

TypeError: ChatPromptTemplate.from_template() missing 1 required positional argument: 'template'

In [19]:
messages

[SystemMessage(content='You are a helpful AI bot'),
 HumanMessage(content='List five ice cream flavors.\nYour response should be a list of comma separated values, eg: `foo, bar, baz`')]

In [33]:
response = chat_model.invoke(messages)


In [34]:
output_parser.parse(response.content)


['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']

In [59]:
output_parser = DatetimeOutputParser()
prompt = """Answer the users question:

{question}

{format_instructions}"""


In [41]:
# 创建一个提示模板，它会基于给定的模板和变量来生成提示
template = ChatPromptTemplate.from_messages(messages=[
    ("system", "You are a helpful AI bot"),
    ("human", prompt),
])

In [42]:
template

ChatPromptTemplate(input_variables=['format_instructions', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful AI bot')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'question'], template='Answer the users question:\n\n{question}\n\n{format_instructions}'))])

In [46]:
messages = template.format_messages(question="What is your name?",format_instructions=output_parser.get_format_instructions())

In [47]:
print(messages)

[SystemMessage(content='You are a helpful AI bot'), HumanMessage(content="Answer the users question:\n\nWhat is your name?\n\nWrite a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1563-12-14T22:50:21.767891Z, 1436-06-02T21:03:14.716728Z, 1793-04-22T21:37:20.840141Z\n\nReturn ONLY this string, no other words!")]


In [49]:
response = chat_model.invoke(messages)

output_parser.parse(response.content)

datetime.datetime(2023, 8, 15, 18, 30, 45, 123456)

In [56]:
prompt = """Answer the users question:

{question}

{format_instructions}"""

template = ChatPromptTemplate.from_template(
    prompt
)

print(template)


input_variables=['format_instructions', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'question'], template='Answer the users question:\n\n{question}\n\n{format_instructions}'))]


In [60]:
template.format_messages(question='around when was bitcoin founded?',format_instructions=output_parser.get_format_instructions())

[HumanMessage(content="Answer the users question:\n\naround when was bitcoin founded?\n\nWrite a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1940-05-10T18:19:18.538399Z, 1830-04-12T18:54:32.408249Z, 0861-08-10T15:21:41.415657Z\n\nReturn ONLY this string, no other words!")]

In [62]:
response = chat_model.invoke(messages)

output_parser.parse(response.content)

datetime.datetime(2022, 11, 15, 18, 30, 45, 123456)

In [72]:

# 创建提示模板
prompt_template = """您是一位专业的鲜花店文案撰写员。
对于售价为 {price} 元的 {flower_name} ，您能提供一个吸引人的简短描述吗？
{format_instructions}"""



# 导入结构化输出解析器和ResponseSchema
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
# 定义我们想要接收的响应模式
response_schemas = [
    ResponseSchema(name="description", description="鲜花的描述文案"),
    ResponseSchema(name="reason", description="问什么要这样写这个文案")
]
# 创建输出解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 获取格式指示
format_instructions = output_parser.get_format_instructions()
# 根据模板创建提示，同时在提示中加入输出解析器的说明
prompt = ChatPromptTemplate.from_template(prompt_template, 
                partial_variables={"format_instructions": format_instructions}) 

# 数据准备
flowers = ["玫瑰", "百合", "康乃馨"]
prices = ["50", "30", "20"]

# 创建一个空的DataFrame用于存储结果
import pandas as pd
df = pd.DataFrame(columns=["flower", "price", "description", "reason"]) # 先声明列名

for flower, price in zip(flowers, prices):
    # 根据提示准备模型的输入
    input = prompt.format(flower_name=flower, price=price)
    
    print(input)


    # 获取模型的输出
    output = chat_model.invoke(input)
    
    print(output)
    
    # 解析模型的输出（这是一个字典结构）
    parsed_output = output_parser.parse(output.content)

    # 在解析后的输出中添加“flower”和“price”
    parsed_output['flower'] = flower
    parsed_output['price'] = price

    # 将解析后的输出添加到DataFrame中
    df.loc[len(df)] = parsed_output  

# 打印字典
print(df.to_dict(orient='records'))

# 保存DataFrame到CSV文件
df.to_csv("flowers_with_descriptions.csv", index=False)

Human: 您是一位专业的鲜花店文案撰写员。
对于售价为 50 元的 玫瑰 ，您能提供一个吸引人的简短描述吗？
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"description": string  // 鲜花的描述文案
	"reason": string  // 问什么要这样写这个文案
}
```
content='```json\n{\n\t"description": "这束粉色玫瑰散发着浪漫的芬芳，是表达爱意和感激的完美选择。",\n\t"reason": "这段描述能够吸引顾客，让他们感受到购买这束玫瑰的浪漫和感动，增加购买欲望。"\n}\n```' response_metadata={'finish_reason': 'stop', 'logprobs': None}
Human: 您是一位专业的鲜花店文案撰写员。
对于售价为 30 元的 百合 ，您能提供一个吸引人的简短描述吗？
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"description": string  // 鲜花的描述文案
	"reason": string  // 问什么要这样写这个文案
}
```
content='```json\n{\n\t"description": "清新优雅的百合，散发着淡淡芬芳，送给心爱的人，传递浪漫与美好。",\n\t"reason": "这样的描述可以让顾客感受到百合的优雅和浪漫之美，吸引他们购买。"\n}\n```' response_metadata={'finish_reason': 'stop', 'logprobs': None}
Human: 您是一位专业的鲜花店文案撰写员。
对于售价为 20 元的 康乃馨 ，您能提供一个吸引人的简短描述吗